# RAG POC 

I combined part of Dexter's and my (Eng Meng's) notebook and optimized certain parts and added in table reading to round out this MVP

In [1]:
# Import libraries
# Standard libraries for reading pdf
import PyPDF2 as pypdf
import PyPDF2
import re
import requests
import io
from io import BytesIO
import concurrent.futures
from itertools import groupby
import random
import string

# Use open parse for reading the tables
import openparse
from bs4 import BeautifulSoup

# Llama libraries for RAG
from llama_index.core import (
    Document,
    Prompt,
    VectorStoreIndex,
    
)
from llama_index.core.schema import TextNode, NodeRelationship, RelatedNodeInfo
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.llama_cpp import LlamaCPP

# For the responses
from llama_index.core.callbacks import (
    CallbackManager,
    LlamaDebugHandler,
    CBEventType,
)
from llama_index.core import ServiceContext
from llama_index.core.text_splitter import SentenceSplitter
from llama_index.core.response_synthesizers import (
    ResponseMode,
    get_response_synthesizer,
)

## 1. Extract and Parse Information

In [2]:
# EM's method
def extract_text_from_page(pdf_reader, pdf_name, index):
    text = pdf_reader.pages[index].extract_text()
    if text:
        text = text.replace('\n', ' ')
    return {'title': f"{pdf_name} page {index + 1}", 'content': text}

def prepare_docs(pdf_docs):
    docs = []
    doc_dict = {}

    for pdf in pdf_docs:
        if isinstance(pdf, BytesIO):
            pdf_name = "in_memory_pdf"
        else:
            pdf_name = pdf

        pdf_reader = PyPDF2.PdfReader(pdf)
        with concurrent.futures.ThreadPoolExecutor() as executor:
            results = list(executor.map(lambda idx: extract_text_from_page(pdf_reader, pdf_name, idx), range(len(pdf_reader.pages))))
        
        docs.extend(results)
        
    for doc in docs:
        title = doc["title"]
        content = doc["content"]
        doc_dict[title] = content

    print("Content and metadata are extracted from the documents")
    return doc_dict

def get_text_chunks(content, metadata):
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=512,
        chunk_overlap=256,
    )
    split_docs = text_splitter.create_documents(content, metadatas=metadata)
    print(f"Documents are split into {len(split_docs)} passages")
    return split_docs

In [3]:
%%time
pdf_docs = ['Orientation_Spring24.pdf']
doc_dicts = prepare_docs(pdf_docs)

Content and metadata are extracted from the documents
CPU times: total: 1.42 s
Wall time: 6.31 s


In [4]:
textNode = []
for key, value in doc_dicts.items():
    txtNode = TextNode(text=value, metadata={"pages": key})
    textNode.append(txtNode)

In [5]:
# Dexters method
# Function to extract content between two given section titles
def extract_all_sections(text):
    # Define the section headers
    headers = [
        "SECTION A. ACADEMIC POLICIES",
        "SECTION B. FOUNDATIONAL COURSE REQUIREMENT",
        "SECTION C. DEGREE REQUIREMENTS",
        "SECTION D. REGISTRATION TIME TICKETS",
        "SECTION E. REGISTRATION HOLDS",
        "SECTION F. REGISTER FOR CLASSES",
        "SECTION G. WAITLISTING FOR A CLOSED COURSE",
        "SECTION H. DROPPING/WITHDRAWING FROM A COURSE(S)",
        "SECTION I. SYSTEMS YOU WILL BE USING AND WHY",
        "SECTION J. IMPORTANT DATES",
        "SECTION K. FREQUENTLY ASKED QUESTIONS (FAQs)",
        "SECTION L. GEORGIA TECH DIRECTORY INFORMATION",
        "SECTION M. GET CONNECTED!",
        "SECTION N. OMSCS ADVISING CONTACT INFORMATION"
    ]
    # Create a dictionary to store the extracted sections
    sections = {}
    # Loop through each header and find the text between current and next header
    for i in range(len(headers)):
        current_header = headers[i]
        # Check if it's the last header
        if (i+1) < len(headers):
            next_header = headers[i+1]
        else:
            next_header = None
        # Find all occurences of current header (match for possible multiple spaces between words)
        current_header_pattern = re.sub(r'\\ ', r'\\s+', re.escape(current_header)) + '\\s+'
        current_indexes = [m.start()+1 for m in re.finditer(current_header_pattern, text, re.IGNORECASE)]
        # Find the latest possible occurence since each header appears multiple times, including in the content page
        for i in range(1, len(current_indexes)):
            last_occurrence_index = current_indexes[-i]
            if next_header:
                # Find all occurences of next header (match for possible multiple spaces between words)
                next_header_pattern = re.sub(r'\\ ', r'\\s+', re.escape(next_header)) + '\\s+'
                next_header_match = re.search(next_header_pattern, text[last_occurrence_index:])
                next_header_index = next_header_match.start() + last_occurrence_index if next_header_match else -1
            else:
                next_header_index = len(text)
            if (last_occurrence_index + len(current_header)) < next_header_index:
                break
        # Extract section text without leading or trailing whitespaces
        section_text = text[last_occurrence_index + len(current_header):next_header_index].strip()
        # Store in the format of <header>: <section text>
        sections[current_header] = current_header + ": " + section_text
    return sections

# Function to read PDF content from PDF file
def read_pdf(file_path):
    # HTTP request to retrieve document
    headers = {"User-Agent": "Mozilla/5.0 (X11; Windows; Windows x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36"}
    r = requests.get(
        url=file_path,
        headers=headers
    )
    # Use PyPDF to read document
    document = io.BytesIO(r.content)
    reader = pypdf.PdfReader(document)
    # Multi page document
    num_pages = len(reader.pages)
    pdf_content = ""
    for i in range(num_pages):
        # Extract each page
        page = reader.pages[i]
        content = page.extract_text()
        # Format document new lines to single spaces
        content = content.replace("\n", " ")
        # Remove footer (page number and instruction)
        content = re.sub(r"\d+$", "", content)
        content = content.replace("Click here to return to Table of Contents", "")
        # Ensure no leading or trailing white spaces
        content = content.strip()
        # Append content with new line delimiter for each page
        pdf_content += content
        pdf_content += "\n"
    return pdf_content

# Function to test reading of orientation document file
def read_orientation_file(url):
    pdf_text = read_pdf(url)
    # Extract all sections
    sections_content = extract_all_sections(pdf_text)
    # Print the extracted content for each section
    for section, content in sections_content.items():
        print(f"{content}\n")
    return sections_content

In [6]:
# Read the 2024 PDF file
url = "https://omscs.gatech.edu/sites/default/files/documents/2024/Orientation%20Documents-Spring%202024%20Orientation%20Document.pdf"
file_2024 = read_orientation_file(url)

file = file_2024
text_embedding_model = "thenlper/gte-base"

textNode = []
for key, value in file.items():
    txtNode = TextNode(text=value, metadata={"section": key})
    textNode.append(txtNode)

SECTION A. ACADEMIC POLICIES: As a Georgia Tech student, you are responsible for knowing these academic policies and information:  ● Academic Catalog  ● Academic Honor Code  ● Student/Faculty Expectations  ● Expectations of Advisors and Advisees

SECTION B. FOUNDATIONAL COURSE REQUIREMENT: 1. You have one calendar year from the time you matriculate to complete the foundational course requirement.  2. If this is your first term as a new admit, you have Spring 2024, Summer 2024, and Fall 2024 to complete the  foundational course requirement.  3. The foundational course requirement is a minimum grade of “B” in two foundational courses.  4. The foundational courses are those with an asterisk (*) listed on the “ Current Courses ” page under the Program  Information tab on the OMSCS website.  5. If at any point during the first three consecutive terms after you matriculate you make less than a “B” in a  foundational course, that term still counts as part of your one-year time frame to comple

## Tables

In [7]:
# Lets extract the tables as well
# pymupdf works better than unitable and is faster in our case!
pdf_docs = 'Orientation_Spring24.pdf'
parser = openparse.DocumentParser(table_args={
    "parsing_algorithm": "pymupdf",
})
parsed_table = parser.parse(pdf_docs)

In [8]:
# For individual tables in a single page we are fine
# However when the tables stretch to other pages, we need to do some creative shizzles
# Get consecutive tables
# Make sure its (text,table),(table),... 
consecutive_tables_list = []
for i in range(len(parsed_table.nodes)):
    if "table" in parsed_table.nodes[i].variant and "table" in parsed_table.nodes[i + 1].variant and not "text" in parsed_table.nodes[i + 1].variant:
        # Grab the consecutive blocks and work on there
        consecutive_tables_list.append(i)
        consecutive_tables_list.append(i+1)
        
consecutive_list = list(set(consecutive_tables_list))


consecutive_groups = []
for k, g in groupby(enumerate(consecutive_list), lambda ix : ix[0] - ix[1]):
    consecutive_groups.append(list(map(lambda x: x[1], g)))

# Create dictionary
consecutive_dict = {index + 1: group for index, group in enumerate(consecutive_groups)}

In [9]:
def extract_table(html):
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('table')
    return table

def extract_rows(table):
    rows = table.find_all('tr')
    return rows

def remove_duplicate_header_rows(html):
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('table')
    if not table:
        return html
    
    # Create a list of all rows in the table
    rows = table.find_all('tr')
    rows_to_remove = []

    # Iterate through the rows starting from the second row
    for i in range(1, len(rows) - 1):
        current_row = rows[i]
        next_row = rows[i + 1]
        
        # Check if the current row and the next row match the pattern
        if current_row.find_all('th') and next_row.find_all('td'):
            current_th_texts = [th.get_text(strip=True) for th in current_row.find_all('th')]
            next_td_texts = [td.get_text(strip=True) for td in next_row.find_all('td')]
            
            if current_th_texts == next_td_texts:
                rows_to_remove.append(current_row)

    # Remove the identified rows
    for row in rows_to_remove:
        row.decompose()

    return str(soup)

def combine_tables(html_dict):
    combined_html_dict = {}
    
    # Iterate over each key-value pair in the dictionary
    for key, indices in html_dict.items():
        # Initialize an empty combined table for the current key
        combined_table = BeautifulSoup('<table border="1"></table>', 'html.parser').table
        # Iterate over each index in the list
        for idx in indices:
            # Extract HTML content for the index
            html_content = parsed_table.nodes[idx].elements[-1].text
            # Extract table from HTML
            table = extract_table(html_content)
            if table:
                # Extract rows from the table
                rows = extract_rows(table)
                # Combine rows into the combined table
                for row in rows:
                    combined_table.append(row)
        # For the combined table we need to parse the headers properly
        # the llm is sensitive to the headers misalignment and stuff
        cleaned_html = remove_duplicate_header_rows(str(combined_table))
        # Store the combined HTML content for the current key
        combined_html_dict[key] = str(cleaned_html)
        #combined_html_dict[key] = str(combined_table)
    
    return combined_html_dict

In [10]:
combined_html_dict = combine_tables(consecutive_dict)

In [11]:
def generate_random_id():
    return ''.join(random.choices(string.ascii_letters + string.digits, k=10))

def create_text_nodes(combined_html_dict):
    text_nodes = []
    for key, html_content in combined_html_dict.items():
        random_id = generate_random_id()
        text_nodes.append(TextNode(text=html_content, id_=random_id))
    return text_nodes

In [12]:
table_nodes = create_text_nodes(combined_html_dict)

In [14]:
# Combine as a list for the different types of nodes
allNodes = textNode + table_nodes

## Load Index and Run the model

In [15]:
text_embedding_model = "thenlper/gte-base"
# Instantiate LLM and embedding model
embed_model = HuggingFaceEmbedding(model_name=text_embedding_model)
#model = 'llama-2-7b-chat.Q4_K_M.gguf'
model = 'ggml-model-Q4_K_M.gguf'

llm = LlamaCPP(model_path=model, temperature=0, max_new_tokens=256, context_window=4096, verbose=True)

C:\Users\UGGTech\anaconda3\envs\RAG_Env\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from ggml-model-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = model
llama_model_loader: - kv   2:                           llama.vocab_size u32              = 128256
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32

In [16]:
llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])
text_splitter = SentenceSplitter(
    chunk_size=512,
    chunk_overlap=256
)
service_context = ServiceContext.from_defaults(
    callback_manager=callback_manager,
    llm=llm,
    embed_model=embed_model,
    text_splitter=text_splitter,
)

C:\Users\UGGTech\AppData\Local\Temp\ipykernel_55764\3092009722.py:7: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [17]:
index = VectorStoreIndex(
    allNodes,
    service_context=service_context,
    embed_model=embed_model,
    llm=llm
)

**********
Trace: index_construction
**********


## Queries

In [18]:
%%time
# Query engine with streaming response and top 1 document filter for efficiency
response_synthesizer = get_response_synthesizer(
    response_mode=ResponseMode.COMPACT,
    service_context=service_context,
    streaming=True
)
query_engine = index.as_query_engine(
    response_synthesizer=response_synthesizer,
    similarity_top_k=1,
    streaming=True
)

CPU times: total: 93.8 ms
Wall time: 148 ms


In [19]:
%%time
response = query_engine.query("What georgia tech courses are equivalent to the OMSCS course CS 6290?")
response.print_response_stream()

 ECE 4100, ECE 6100, and CS 4290.


llama_print_timings:        load time =   48012.75 ms
llama_print_timings:      sample time =      20.46 ms /    19 runs   (    1.08 ms per token,   928.60 tokens per second)
llama_print_timings: prompt eval time =  208184.13 ms /  1994 tokens (  104.41 ms per token,     9.58 tokens per second)
llama_print_timings:        eval time =    5381.96 ms /    18 runs   (  299.00 ms per token,     3.34 tokens per second)
llama_print_timings:       total time =  213716.59 ms /  2012 tokens


CPU times: total: 23min 45s
Wall time: 3min 33s


In [20]:
%%time
response = query_engine.query("What georgia tech courses are equivalent to the OMSCS course CS 7650?")
response.print_response_stream()

Llama.generate: prefix-match hit


 CS 4650 is equivalent to the OMSCS course CS 7650 at Georgia Tech.


llama_print_timings:        load time =   48012.75 ms
llama_print_timings:      sample time =      20.46 ms /    21 runs   (    0.97 ms per token,  1026.24 tokens per second)
llama_print_timings: prompt eval time =    2013.37 ms /     7 tokens (  287.62 ms per token,     3.48 tokens per second)
llama_print_timings:        eval time =    5817.14 ms /    20 runs   (  290.86 ms per token,     3.44 tokens per second)
llama_print_timings:       total time =    7941.25 ms /    27 tokens


CPU times: total: 21.6 s
Wall time: 8.14 s


In [21]:
%%time 
response = query_engine.query("What georgia tech courses are equivalent to the OMSCS course CS 7646?")
response.print_response_stream()

Llama.generate: prefix-match hit


 CS 4646 and CS 4803 Machine Learning for Trading.


llama_print_timings:        load time =   48012.75 ms
llama_print_timings:      sample time =      15.28 ms /    15 runs   (    1.02 ms per token,   982.00 tokens per second)
llama_print_timings: prompt eval time =    1011.04 ms /     7 tokens (  144.43 ms per token,     6.92 tokens per second)
llama_print_timings:        eval time =    3882.34 ms /    14 runs   (  277.31 ms per token,     3.61 tokens per second)
llama_print_timings:       total time =    4965.73 ms /    21 tokens


CPU times: total: 13 s
Wall time: 5.06 s


In [22]:
%%time
response = query_engine.query("When does the first day of class of spring semester 2024 start?")
response.print_response_stream()

Llama.generate: prefix-match hit


8 January 2024.


llama_print_timings:        load time =   48012.75 ms
llama_print_timings:      sample time =       6.67 ms /     7 runs   (    0.95 ms per token,  1048.69 tokens per second)
llama_print_timings: prompt eval time =   32546.56 ms /   320 tokens (  101.71 ms per token,     9.83 tokens per second)
llama_print_timings:        eval time =    1489.25 ms /     6 runs   (  248.21 ms per token,     4.03 tokens per second)
llama_print_timings:       total time =   34069.26 ms /   326 tokens


CPU times: total: 3min 54s
Wall time: 34.2 s


In [23]:
%%time
response = query_engine.query("Is there orientation for OMSCS?")
response.print_response_stream()

Llama.generate: prefix-match hit


1. Yes, there is an orientation for OMSCS students.
2. The email address to contact for all OMSCS Advising questions is oms-advising@cc.gatech.edu.
3. When sending emails, include your full name and nine-digit GTID number (90XXXXXXX), replacing the first two digits with “xx”.
4. After sending an email, you will receive an automated response with a ticket number confirming that your message has been received.
5. You should receive two emails: one with the response from OMSCS Advising and another stating that your ticket has been resolved.
6. If you have further questions after receiving a response, create a new ticket by sending a separate email to oms-advising@cc.gatech.edu.
7. Do not copy other people/departments on the message – OMSCS Advising will direct you to another department if necessary.
8. Please allow 24-48 business hours as a standard response time, and during certain times of the year, such as registration and the end of the term, the response time may increase.
9. Do not 


llama_print_timings:        load time =   48012.75 ms
llama_print_timings:      sample time =     250.29 ms /   256 runs   (    0.98 ms per token,  1022.81 tokens per second)
llama_print_timings: prompt eval time =   84605.16 ms /   717 tokens (  118.00 ms per token,     8.47 tokens per second)
llama_print_timings:        eval time =   68649.11 ms /   255 runs   (  269.21 ms per token,     3.71 tokens per second)
llama_print_timings:       total time =  154676.02 ms /   972 tokens


CPU times: total: 10min 23s
Wall time: 2min 34s


In [24]:
%time
response = query_engine.query("When will the grades be available for spring 2024?")
response.print_response_stream()

CPU times: total: 0 ns
Wall time: 0 ns


Llama.generate: prefix-match hit


 The grades will be available on May 7 after 6:00pm ET for Spring 2024.


llama_print_timings:        load time =   48012.75 ms
llama_print_timings:      sample time =      22.60 ms /    23 runs   (    0.98 ms per token,  1017.61 tokens per second)
llama_print_timings: prompt eval time =   33768.46 ms /   314 tokens (  107.54 ms per token,     9.30 tokens per second)
llama_print_timings:        eval time =    5645.79 ms /    22 runs   (  256.63 ms per token,     3.90 tokens per second)
llama_print_timings:       total time =   39537.27 ms /   336 tokens


In [25]:
%%time
response = query_engine.query("What georgia tech courses are equivalent to the OMSCS course CS 6290?")
response.print_response_stream()

Llama.generate: prefix-match hit


 ECE 4100, ECE 6100, and CS 4290.


llama_print_timings:        load time =   48012.75 ms
llama_print_timings:      sample time =      25.77 ms /    19 runs   (    1.36 ms per token,   737.32 tokens per second)
llama_print_timings: prompt eval time =  235710.94 ms /  1984 tokens (  118.81 ms per token,     8.42 tokens per second)
llama_print_timings:        eval time =    6590.25 ms /    18 runs   (  366.13 ms per token,     2.73 tokens per second)
llama_print_timings:       total time =  242454.08 ms /  2002 tokens


CPU times: total: 26min 50s
Wall time: 4min 2s


In [26]:
%%time
response = query_engine.query("During the spring semester what is the maximum number of hours i can enroll for?")
response.print_response_stream()

Llama.generate: prefix-match hit


7 hours.


llama_print_timings:        load time =   48012.75 ms
llama_print_timings:      sample time =       4.25 ms /     4 runs   (    1.06 ms per token,   940.51 tokens per second)
llama_print_timings: prompt eval time =  198262.28 ms /  1711 tokens (  115.88 ms per token,     8.63 tokens per second)
llama_print_timings:        eval time =     876.52 ms /     3 runs   (  292.17 ms per token,     3.42 tokens per second)
llama_print_timings:       total time =  199163.72 ms /  1714 tokens


CPU times: total: 22min 43s
Wall time: 3min 19s


In [27]:
%%time
response = query_engine.query("What georgia tech courses are equivalent to the OMSCS course CS 6300?")
response.print_response_stream()

Llama.generate: prefix-match hit


 CS 6144 and CS 8803 Advanced Software Engineering.


llama_print_timings:        load time =   48012.75 ms
llama_print_timings:      sample time =      23.78 ms /    14 runs   (    1.70 ms per token,   588.83 tokens per second)
llama_print_timings: prompt eval time =  250989.41 ms /  1984 tokens (  126.51 ms per token,     7.90 tokens per second)
llama_print_timings:        eval time =    6911.48 ms /    13 runs   (  531.65 ms per token,     1.88 tokens per second)
llama_print_timings:       total time =  258081.85 ms /  1997 tokens


CPU times: total: 27min 27s
Wall time: 4min 18s


In [28]:
%%time
response = query_engine.query("How many hours can I enroll for in Spring?")
response.print_response_stream()

Llama.generate: prefix-match hit


12 hours. 
Note: The maximum number of hours that a student can enroll for is typically determined by their academic program or department. In this case, it is not specified what the maximum enrollment limit is for OMSCS students at Georgia Tech. However, most universities have a general policy that allows undergraduate students to take up to 18 credit hours per semester and graduate students to take up to 12 credit hours per semester. It's always best to check with your academic advisor or department for specific policies regarding maximum enrollment limits.


llama_print_timings:        load time =   48012.75 ms
llama_print_timings:      sample time =     136.79 ms /   107 runs   (    1.28 ms per token,   782.22 tokens per second)
llama_print_timings: prompt eval time =   38835.73 ms /   259 tokens (  149.94 ms per token,     6.67 tokens per second)
llama_print_timings:        eval time =   41129.91 ms /   106 runs   (  388.02 ms per token,     2.58 tokens per second)
llama_print_timings:       total time =   80971.61 ms /   365 tokens


CPU times: total: 4min 21s
Wall time: 1min 21s


In [29]:
%%time
response = query_engine.query("What happens if i drop before the withdrawal deadline?")
response.print_response_stream()

Llama.generate: prefix-match hit


 If you drop a course, this action can only be done during periods of registration. For Spring 2024 registration, the last day to drop a course is Friday, January 12 by 4:00pm ET. If a student drops a course, the course will not be reflected on the student’s transcript. It will appear as if the student never registered for the course, according to the transcript, so it will NOT count as a record of enrollment. If a student withdraws from a course (this is done after Spring 2024 registration has ended and before the semester’s withdrawal deadline), this action will result in a “W” on the student’s transcript. If a student withdraws from a course (or courses) and receives a “W” on the transcript, this counts as a record of enrollment for that specific term. For step-by-step instructions on how to drop/withdraw from a course(s), please visit the Registrar’


llama_print_timings:        load time =   48012.75 ms
llama_print_timings:      sample time =     215.88 ms /   192 runs   (    1.12 ms per token,   889.37 tokens per second)
llama_print_timings: prompt eval time =  164541.47 ms /  1291 tokens (  127.45 ms per token,     7.85 tokens per second)
llama_print_timings:        eval time =   68650.21 ms /   191 runs   (  359.43 ms per token,     2.78 tokens per second)
llama_print_timings:       total time =  234712.93 ms /  1482 tokens


CPU times: total: 18min 30s
Wall time: 3min 55s


In [30]:
%%time
response = query_engine.query("When can i register for spring semester 2024?")
response.print_response_stream()

Llama.generate: prefix-match hit


 Registration for Spring 2024 begins on January 4 and ends on January 12 at 4:00pm ET. Please check your time ticket to see the exact day and time you can begin registering. Remember that all times and deadlines are based on the Eastern time zone, and registration is first come, first served, so you will want to register as soon as your time ticket opens.


llama_print_timings:        load time =   48012.75 ms
llama_print_timings:      sample time =      76.65 ms /    80 runs   (    0.96 ms per token,  1043.66 tokens per second)
llama_print_timings: prompt eval time =  205788.10 ms /  1703 tokens (  120.84 ms per token,     8.28 tokens per second)
llama_print_timings:        eval time =   22790.43 ms /    79 runs   (  288.49 ms per token,     3.47 tokens per second)
llama_print_timings:       total time =  228986.96 ms /  1782 tokens


CPU times: total: 23min 28s
Wall time: 3min 49s


In [31]:
%%time
response = query_engine.query("How many courses should i take in a semester if i am starting out?")
response.print_response_stream()

Llama.generate: prefix-match hit


1-2 courses per semester is recommended for new admits to ensure they can focus on their foundational course requirements without overloading themselves with too much coursework at once. It's important to remember that you have one calendar year from the time you matriculate to complete the foundational course requirement, so taking a manageable number of courses each term will help you stay on track and meet this deadline. Additionally, if you are starting out in Spring 2024, you will be restricted to enrolling only in foundational courses until you have satisfied the foundational course requirement. This means that it's best to focus on completing these two courses within your first three consecutive terms after matriculating.


llama_print_timings:        load time =   48012.75 ms
llama_print_timings:      sample time =     136.20 ms /   135 runs   (    1.01 ms per token,   991.18 tokens per second)
llama_print_timings: prompt eval time =   44952.14 ms /   479 tokens (   93.85 ms per token,    10.66 tokens per second)
llama_print_timings:        eval time =   37870.45 ms /   134 runs   (  282.62 ms per token,     3.54 tokens per second)
llama_print_timings:       total time =   83572.20 ms /   613 tokens


CPU times: total: 5min 52s
Wall time: 1min 23s


In [32]:
%%time
response = query_engine.query("Can you tell me what degree specializations can i pick as an OMSCS student?")
response.print_response_stream()

Llama.generate: prefix-match hit


 As an OMSCS student, you can choose from the following areas of specialization:
1. Computational Perception and Robotics
2. Computing Systems
3. Human-Computer Interaction
4. Interactive Intelligence
5. Machine Learning


llama_print_timings:        load time =   48012.75 ms
llama_print_timings:      sample time =      46.19 ms /    46 runs   (    1.00 ms per token,   995.95 tokens per second)
llama_print_timings: prompt eval time =   54782.16 ms /   562 tokens (   97.48 ms per token,    10.26 tokens per second)
llama_print_timings:        eval time =   12392.93 ms /    45 runs   (  275.40 ms per token,     3.63 tokens per second)
llama_print_timings:       total time =   67428.73 ms /   607 tokens


CPU times: total: 6min 21s
Wall time: 1min 7s


In [33]:
%%time
response = query_engine.query("What advice can you give me to best succeed in OMSCS?")
response.print_response_stream()

Llama.generate: prefix-match hit


1. Stay organized and manage your time effectively.
2. Actively participate in class discussions and engage with course materials.
3. Utilize resources such as the OMSCS Advising team for support and guidance.
4. Connect with other students through online communities or social media groups.
5. Set realistic goals and milestones to track your progress throughout the program.
6. Take advantage of opportunities for networking, professional development, and career advancement.
7. Stay motivated and maintain a positive attitude throughout the program.
8. Seek help when needed from instructors, advisors, or peers.
9. Continuously learn and adapt to new technologies and industry trends.
10. Celebrate your achievements along the way and enjoy the learning journey!


llama_print_timings:        load time =   48012.75 ms
llama_print_timings:      sample time =     147.40 ms /   145 runs   (    1.02 ms per token,   983.72 tokens per second)
llama_print_timings: prompt eval time =   76265.57 ms /   723 tokens (  105.48 ms per token,     9.48 tokens per second)
llama_print_timings:        eval time =   41608.78 ms /   144 runs   (  288.95 ms per token,     3.46 tokens per second)
llama_print_timings:       total time =  118950.00 ms /   867 tokens


CPU times: total: 9min 27s
Wall time: 1min 59s


In [34]:
%%time
response = query_engine.query("What happens if I did not pay for the semester, will i get a W in my transcript?")
response.print_response_stream()

Llama.generate: prefix-match hit


 If you do not pay your tuition/fees in full by the fee payment deadline, you will be subject to class cancellation and holds. If your schedule is canceled, you will be notified by email. Appeals can be submitted by visiting the Bursar’s website . If you are reinstated into classes, a reinstatement fee of $200.00 will apply, and the account balance must be paid immediately to avoid re-cancellation. Please note: Reinstatement does not guarantee that faculty will allow students to make up any work missed. It is the student’s responsibility to confirm ahead of being reinstated what accommodations, if any, will be permitted during the time frame in which the student was not enrolled due to non-payment. If you withdraw from a course, you will receive a “W”. A “W” is visible on your transcript, but it has no impact on your GPA. If you retake a course from which you previously have withdrawn, the “W” from your first attempt and the new grade will appear on your transcript. Therefore, if you d


llama_print_timings:        load time =   48012.75 ms
llama_print_timings:      sample time =     257.36 ms /   244 runs   (    1.05 ms per token,   948.10 tokens per second)
llama_print_timings: prompt eval time =  142378.55 ms /  1301 tokens (  109.44 ms per token,     9.14 tokens per second)
llama_print_timings:        eval time =   73336.63 ms /   243 runs   (  301.80 ms per token,     3.31 tokens per second)
llama_print_timings:       total time =  217235.33 ms /  1544 tokens


CPU times: total: 17min 42s
Wall time: 3min 37s


In [35]:
%%time
response = query_engine.query("What time zone is this document based on?")
response.print_response_stream()

Llama.generate: prefix-match hit


 The document is based on Eastern Time (ET).


llama_print_timings:        load time =   48012.75 ms
llama_print_timings:      sample time =      11.45 ms /    11 runs   (    1.04 ms per token,   960.78 tokens per second)
llama_print_timings: prompt eval time =   28151.62 ms /   255 tokens (  110.40 ms per token,     9.06 tokens per second)
llama_print_timings:        eval time =    2850.83 ms /    10 runs   (  285.08 ms per token,     3.51 tokens per second)
llama_print_timings:       total time =   31098.13 ms /   265 tokens


CPU times: total: 3min 5s
Wall time: 31.2 s


In [36]:
%%time
response = query_engine.query("What does O and 1 mean in courses?")
response.print_response_stream()

Llama.generate: prefix-match hit


0 and 1 refer to the grading scale used by Georgia Tech for their online Master of Science in Computer Science (OMSCS) program. In this grading system, a grade of "0" represents a failing grade, while a grade of "1" represents a passing grade. This grading system is unique to OMSCS and may differ from other programs or institutions that use different grading scales. It's important for students to understand the grading scale used in their program as it can impact their academic progress and overall performance.


llama_print_timings:        load time =   48012.75 ms
llama_print_timings:      sample time =     110.80 ms /   106 runs   (    1.05 ms per token,   956.65 tokens per second)
llama_print_timings: prompt eval time =  150256.14 ms /  1291 tokens (  116.39 ms per token,     8.59 tokens per second)
llama_print_timings:        eval time =   34262.15 ms /   105 runs   (  326.31 ms per token,     3.06 tokens per second)
llama_print_timings:       total time =  185178.96 ms /  1396 tokens


CPU times: total: 17min 14s
Wall time: 3min 5s


In [37]:
%%time
response = query_engine.query("Why is my class cancelled?")
response.print_response_stream()

Llama.generate: prefix-match hit


 If you do not pay your tuition/fees in full by the fee payment deadline, you will be subject to class cancellation and holds. If your schedule is canceled, you will be notified by email. Appeals can be submitted by visiting the Bursar’s website . If you are reinstated into classes, a reinstatement fee of $200.00 will apply, and the account balance must be paid immediately to avoid re-cancellation. Please note: Reinstatement does not guarantee that faculty will allow students to make up any work missed. It is the student’s responsibility to confirm ahead of being reinstated what accommodations, if any, will be permitted during the time frame in which the student was not enrolled due to non-payment.


llama_print_timings:        load time =   48012.75 ms
llama_print_timings:      sample time =     160.29 ms /   148 runs   (    1.08 ms per token,   923.31 tokens per second)
llama_print_timings: prompt eval time =    2140.54 ms /    10 tokens (  214.05 ms per token,     4.67 tokens per second)
llama_print_timings:        eval time =   52589.60 ms /   147 runs   (  357.75 ms per token,     2.80 tokens per second)
llama_print_timings:       total time =   55726.54 ms /   157 tokens


CPU times: total: 55.7 s
Wall time: 55.9 s


In [38]:
%%time
response = query_engine.query("How can I connect with other students?")
response.print_response_stream()

Llama.generate: prefix-match hit


1. You can connect with other students through social media platforms such as Facebook (OMSCS Official), Twitter (OMSCS Official), OMSCS reddit, and OMSCS MeWe. These platforms are not official OMSCS pages but are monitored by OMSCS employees periodically. They are run by students for students and provide a great way to connect with classmates from around the world. 2. Engage in discussions on these platforms, share your experiences, ask questions, and offer help when needed. This will help you build relationships with other students and create a supportive learning community.


llama_print_timings:        load time =   48012.75 ms
llama_print_timings:      sample time =     119.96 ms /   121 runs   (    0.99 ms per token,  1008.69 tokens per second)
llama_print_timings: prompt eval time =   19781.58 ms /   173 tokens (  114.34 ms per token,     8.75 tokens per second)
llama_print_timings:        eval time =   34679.69 ms /   120 runs   (  289.00 ms per token,     3.46 tokens per second)
llama_print_timings:       total time =   55275.96 ms /   293 tokens


CPU times: total: 2min 38s
Wall time: 55.4 s


In [39]:
%%time
response = query_engine.query("Can I take classes after I have graduated from the OMSCS program?")
response.print_response_stream()

Llama.generate: prefix-match hit


 Based on the provided context information, there is no mention of taking classes after graduating from the OMSCS program. The information focuses on the degree requirements for students currently enrolled in the program. Therefore, it is not possible to determine if you can take classes after graduating based solely on this context information. It would be best to refer to Georgia Tech's official policies or contact their admissions office for more detailed information regarding post-graduation course enrollment.


llama_print_timings:        load time =   48012.75 ms
llama_print_timings:      sample time =      99.68 ms /    90 runs   (    1.11 ms per token,   902.92 tokens per second)
llama_print_timings: prompt eval time =   69044.60 ms /   559 tokens (  123.51 ms per token,     8.10 tokens per second)
llama_print_timings:        eval time =   29117.93 ms /    89 runs   (  327.17 ms per token,     3.06 tokens per second)
llama_print_timings:       total time =   98799.04 ms /   648 tokens


CPU times: total: 7min 35s
Wall time: 1min 38s


In [40]:
%%time
response = query_engine.query("I am skipping the summer semester, do i need to do any administrative action to enroll for fall semester?")
response.print_response_stream()

Llama.generate: prefix-match hit


1. Yes, you should check if there are any holds on your account that might prevent registration for the Fall semester.
2. If you have a hold on your account, you must clear it before registering for classes. Contact the department who placed the hold as they can remove it.
3. Ensure all required official transcripts and documents are submitted to avoid future holds.
4. As a distance learner, you do not need to complete immunization requirements.
5. Clear any Financial Agreement holds in OSCAR by selecting "Financial Responsibility Agreement" on the Registration menu, reading the agreement, entering your initials, and clicking "Submit".
6. Clear any Registrar Location holds several days prior to your active time ticket in OSCAR by selecting "Term Study Location Identification", choosing your location from the pull-down menu, and then clicking "Submit".


llama_print_timings:        load time =   48012.75 ms
llama_print_timings:      sample time =     181.10 ms /   167 runs   (    1.08 ms per token,   922.16 tokens per second)
llama_print_timings: prompt eval time =   98283.02 ms /   650 tokens (  151.20 ms per token,     6.61 tokens per second)
llama_print_timings:        eval time =   55335.81 ms /   166 runs   (  333.35 ms per token,     3.00 tokens per second)
llama_print_timings:       total time =  154818.21 ms /   816 tokens


CPU times: total: 10min 7s
Wall time: 2min 34s


In [41]:
%%time
response = query_engine.query("Do I get W on my transcript if I do not pay for the course?")
response.print_response_stream()

Llama.generate: prefix-match hit


 No, you will not receive a "W" on your transcript if you do not pay for the course. If you do not pay your tuition/fees in full by the fee payment deadline, you will be subject to class cancellation and holds. If your schedule is canceled, you will be notified by email. Appeals can be submitted by visiting the Bursar’s website. If you are reinstated into classes, a reinstatement fee of $200.00 will apply, and the account balance must be paid immediately to avoid re-cancellation. Please note: Reinstatement does not guarantee that faculty will allow students to make up any work missed. It is the student's responsibility to confirm ahead of being reinstated what accommodations, if any, will be permitted during the time frame in which the student was not enrolled due to non-payment.


llama_print_timings:        load time =   48012.75 ms
llama_print_timings:      sample time =     178.12 ms /   170 runs   (    1.05 ms per token,   954.44 tokens per second)
llama_print_timings: prompt eval time =  172385.00 ms /  1297 tokens (  132.91 ms per token,     7.52 tokens per second)
llama_print_timings:        eval time =   50388.85 ms /   169 runs   (  298.16 ms per token,     3.35 tokens per second)
llama_print_timings:       total time =  223736.41 ms /  1466 tokens


CPU times: total: 18min 21s
Wall time: 3min 43s


In [42]:
%%time
response = query_engine.query("Can OMSCS students get a buzzcard?")
response.print_response_stream()

Llama.generate: prefix-match hit


 Based on the provided context information, there is no mention of OMSCS students being able to obtain a BuzzCard. It is recommended to reach out to the official OMSCS channels or contact their support team for more detailed information regarding this matter.


llama_print_timings:        load time =   48012.75 ms
llama_print_timings:      sample time =      38.94 ms /    51 runs   (    0.76 ms per token,  1309.64 tokens per second)
llama_print_timings: prompt eval time =   22050.40 ms /   175 tokens (  126.00 ms per token,     7.94 tokens per second)
llama_print_timings:        eval time =   16120.04 ms /    50 runs   (  322.40 ms per token,     3.10 tokens per second)
llama_print_timings:       total time =   38969.81 ms /   225 tokens


CPU times: total: 1min 46s
Wall time: 40.2 s


In [43]:
%%time
response = query_engine.query("Am i eligible to get buzzcard if i am an OMSCS student?")
response.print_response_stream()

Llama.generate: prefix-match hit


 Yes, you are eligible to get a BuzzCard as an OMSCS student. You can obtain your BuzzCard by visiting the BuzzCard office in person or by mail-in order form. For more information on how to obtain a BuzzCard, please visit the following webpage: <https://www.cc.gatech.edu/buzzcard>. Additionally, you may contact the BuzzCard office at buzzcard@cc.gatech.edu for further assistance. Remember to provide your GT ID number when requesting a BuzzCard.


llama_print_timings:        load time =   48012.75 ms
llama_print_timings:      sample time =      85.52 ms /   104 runs   (    0.82 ms per token,  1216.10 tokens per second)
llama_print_timings: prompt eval time =   59794.28 ms /   725 tokens (   82.47 ms per token,    12.12 tokens per second)
llama_print_timings:        eval time =   26521.91 ms /   103 runs   (  257.49 ms per token,     3.88 tokens per second)
llama_print_timings:       total time =   87262.78 ms /   828 tokens


CPU times: total: 7min 27s
Wall time: 1min 28s


In [45]:
%%time
response = query_engine.query("How long do I have to wait for Advising to reply?")
response.print_response_stream()

Llama.generate: prefix-match hit


24-48 business hours as a standard response time. During certain times of the year, such as registration and the end of the term, the response time may increase. We greatly appreciate your patience and understanding.


llama_print_timings:        load time =   48012.75 ms
llama_print_timings:      sample time =      33.68 ms /    43 runs   (    0.78 ms per token,  1276.61 tokens per second)
llama_print_timings: prompt eval time =   72650.54 ms /   722 tokens (  100.62 ms per token,     9.94 tokens per second)
llama_print_timings:        eval time =   11525.24 ms /    42 runs   (  274.41 ms per token,     3.64 tokens per second)
llama_print_timings:       total time =   84550.13 ms /   764 tokens


CPU times: total: 7min 28s
Wall time: 1min 24s


In [46]:
%%time
response = query_engine.query("Would you advise on doing cs7641 as a first course in OMSCS?")
response.print_response_stream()

Llama.generate: prefix-match hit


 CS7641 is an introductory course to Artificial Intelligence (AI) that covers topics such as search algorithms, game playing, machine learning, and expert systems. It is not explicitly mentioned if it is recommended as the first course for the OMSCS program. However, since it is an introductory course in AI, it could be a good starting point for students who have little to no background in computer science or AI. It would depend on the student's prior knowledge and experience in these areas. If you are new to CS/AI, then CS7641 might be a suitable first course. However, if you already have some background in CS/AI, it may not be the best starting point as there might be more advanced courses that could better suit your needs. It is always recommended to consult with an academic advisor or review the OMSCS curriculum to determine the most appropriate sequence of courses for your individual situation.


llama_print_timings:        load time =   48012.75 ms
llama_print_timings:      sample time =     148.07 ms /   184 runs   (    0.80 ms per token,  1242.69 tokens per second)
llama_print_timings: prompt eval time =   62760.12 ms /   561 tokens (  111.87 ms per token,     8.94 tokens per second)
llama_print_timings:        eval time =   51502.97 ms /   183 runs   (  281.44 ms per token,     3.55 tokens per second)
llama_print_timings:       total time =  116887.07 ms /   744 tokens


CPU times: total: 7min 25s
Wall time: 1min 57s


In [48]:
%%time
response = query_engine.query("How many ISYEs courses are we allowed to take if we’re still taking the appropriate amount of CSEs? ")
response.print_response_stream()

Llama.generate: prefix-match hit


6 hours may be taken at the 4000-level and/or with a subject code other than CS or CSE. Therefore, you can take up to six ISYE courses while still fulfilling the degree requirements for OMSCS.


llama_print_timings:        load time =   48012.75 ms
llama_print_timings:      sample time =      35.14 ms /    47 runs   (    0.75 ms per token,  1337.55 tokens per second)
llama_print_timings: prompt eval time =    2125.11 ms /    25 tokens (   85.00 ms per token,    11.76 tokens per second)
llama_print_timings:        eval time =   12232.70 ms /    46 runs   (  265.93 ms per token,     3.76 tokens per second)
llama_print_timings:       total time =   14562.41 ms /    71 tokens


CPU times: total: 30.8 s
Wall time: 14.7 s


In [49]:
%%time
response = query_engine.query("How many course hours does one full course in OMSCS have")
response.print_response_stream()

Llama.generate: prefix-match hit


3 credit hours per course.


llama_print_timings:        load time =   48012.75 ms
llama_print_timings:      sample time =       4.91 ms /     7 runs   (    0.70 ms per token,  1426.82 tokens per second)
llama_print_timings: prompt eval time =    1574.55 ms /    15 tokens (  104.97 ms per token,     9.53 tokens per second)
llama_print_timings:        eval time =    1497.08 ms /     6 runs   (  249.51 ms per token,     4.01 tokens per second)
llama_print_timings:       total time =    3119.02 ms /    21 tokens


CPU times: total: 12.9 s
Wall time: 3.22 s


In [50]:
%%time
response = query_engine.query("Is it possible to take 3 classes in the first semester?")
response.print_response_stream()

Llama.generate: prefix-match hit


1. Yes, it is possible to take 3 classes in the first semester.
2. However, it is important to note that you must ensure you meet the foundational course requirement within the one-year time frame as outlined in Section B of the context information provided.


llama_print_timings:        load time =   48012.75 ms
llama_print_timings:      sample time =      39.59 ms /    54 runs   (    0.73 ms per token,  1364.12 tokens per second)
llama_print_timings: prompt eval time =   34682.04 ms /   477 tokens (   72.71 ms per token,    13.75 tokens per second)
llama_print_timings:        eval time =   13718.00 ms /    53 runs   (  258.83 ms per token,     3.86 tokens per second)
llama_print_timings:       total time =   48865.20 ms /   530 tokens


CPU times: total: 4min 22s
Wall time: 49 s
